In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import math
import shapely as shp
from datetime import datetime
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import re
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Notes: current status is informative, but program is also informative

### perhaps add a feature that is the number of occupied properties within a buffer of every property - alice will send over parcel_ID lat/long data

## Notes 2-1-17
1. add MCM condition to dummy variables ###CHECK
2. add any houses that score very high on the occupancy model as homes that should not be demolished ###CHECK
3. update DFD fire list
4. figure out regularization techniques for logit "regression" model ###check
5. run accuracy scores / 'mse' ###check

In [2]:
wd = os.getcwd()
os.chdir('../data')
os.getcwd()

'/Users/geoffperrin/Desktop/DLBA/demo_project/data'

In [3]:
#read in detroit feature dataset
city_df = pd.read_csv('CitywideIndicators_20170203.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## build labeled training dataset

In [4]:
city_df.columns

Index([u'Account ID', u'Parcel ID', u'Account Name', u'Property Ownership',
       u'Property Class', u'Program', u'Current Status',
       u'Non-City Demo Program', u'Current Status Date', u'Neighborhood',
       u'Neighborhood Sales Median', u'DFD Fire: Occurrence Date',
       u'USPS: Vacancy Indicated', u'USPS: Last Update',
       u'Occupancy Model: Probability Pct', u'Occupancy Model: Last Update',
       u'DNG Number', u'Foreclosure Year', u'WCT Foreclosure Status',
       u'MCM Blext Created At', u'MCM Structure', u'MCM Occupancy', u'MCM Use',
       u'MCM Units', u'MCM Residence Type', u'MCM Commercial Use',
       u'MCM Industrial Use', u'MCM Institutional Use', u'MCM Public Use',
       u'MCM Condition', u'MCM Fire Damage', u'MCM Level of Fire Damage',
       u'MCM Boarding Needed', u'MCM Trespass', u'MCM Vacant Lot',
       u'MCM Maintained', u'MCM Dumping'],
      dtype='object')

In [5]:
#only keep DLBA Owned properties for the training data OR with a very high occupancy score
no_demo_occupancy_thresh = 95.0
dlba_df = city_df[(city_df['Property Ownership'] == 'DLBA Owned') | \
                  (city_df['Occupancy Model: Probability Pct'] > no_demo_occupancy_thresh)]

#remove lots
dlba_df = dlba_df[dlba_df['Property Class'] != 'Residential-Lot']

#only keep residential lots
dlba_df = dlba_df[dlba_df['Property Class'] == 'Residential-Structure']


dlba_df_small = dlba_df.rename(columns=lambda x: x.replace(' ', '_'))
dlba_df_small.head()

,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
0,001j000000KoyiZ,16032844.,16132 Cherrylawn,Formerly DLBA Owned,Residential-Structure,Auction,Compliance Achieved (Sales),NaN,9/13/2016,Fitzgerald/Marygrove,...,NaN,NaN,Fair,No,NaN,NaN,Secure,NaN,NaN,No
2,001j000000Idx55,22079053.,17175 Faust,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Evergreen-Outer Drive,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
5,001j000000Idvpv,22080009.,17620 Greenview,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Evergreen-Outer Drive,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
6,001j000000IdzVm,22064522.,15359 Biltmore,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Crary/St Marys,...,NaN,NaN,Fair,No,NaN,No,Secure,NaN,NaN,No
10,001j000000Iem9O,21064817.,11080 Lakepointe,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Yorkshire Woods,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No


## need to fix nans in neighborhood sales median
### all the neighborhoods should be in there, but some rows have NaNs!

In [7]:
dlba_df_small[pd.isnull(dlba_df_small.Neighborhood_Sales_Median)].head(10)

,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
117,001j000000If17Z,13000506.,3128 Woods Circle,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Elmwood Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
360,001j000000If1GO,13000513.,3142 Woods Circle,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Elmwood Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
485,001j000000IeUIA,17000010.002L,8330 E Jefferson,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Gold Coast,...,NaN,NaN,Good,NaN,NaN,No,NaN,NaN,NaN,No
765,001j000000If1Gd,13000529.001,3190 Woods Circle,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Elmwood Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
1413,001j000000If1RM,13000493.,3102 Woods Circle,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Elmwood Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
1467,001j000000If1RN,13000494.,3104 Woods Circle,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Elmwood Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
2418,001j000000IeKPl,22119626.005,9145 West Parkway,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Rouge Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
2631,001j000000Ie3jR,21063980.,4181 Wayburn,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Morningside,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
2688,001j000000IeKPv,22119626.004,9155 West Parkway,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Rouge Park,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
2883,001j000000Iechp,13012559.,19405 Gable,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,NaN,5/2/2016,Farwell,...,NaN,NaN,Suggest Demolition,Yes,Collapsed,Yes,Open,NaN,NaN,No


In [8]:
dlba_df_neighborhood = dlba_df_small[['Neighborhood', 'Neighborhood_Sales_Median']]
dlba_df_neighborhood.groupby(['Neighborhood'], as_index=False).mean().head()

,Neighborhood,Neighborhood_Sales_Median
0,Airport Sub,0.57
1,Arden Park,NaN
2,Aviation Sub,6.24
3,Bagley,11.68
4,Barton-McFarland,1.24


In [9]:
#read in detroit parcel distance to school df
school_df = pd.read_csv('DistanceToSchool_20170123.csv')
school_df_small = school_df[["PARCELNO", "DistanceToSchool_FT"]]
school_df_small.columns = ['Parcel_ID', 'dist_to_school']
school_df_small.head()

,Parcel_ID,dist_to_school
0,16006256.,2196.731029
1,04000103-4,2375.448135
2,04003341.,2442.534185
3,04003339-40,2510.715691
4,04000119.,2341.253487


In [10]:
#merge distance to school to city feature df
dlba_school_df = dlba_df_small.merge(school_df_small, on='Parcel_ID')
len(dlba_school_df)

153837

In [12]:
#code up demo decision yes / no
demo_decision_dict = {'Demo Awaiting Funding': 1,
        'Demo Pipeline': 1,
        'Demo Contracted': 1,
        'Demo Review': 1,
        'Demolished': 999,
        'Construction Complete': 0,
        'Occupied': 0,
        'Sold': 0,
        'Institutional Partner Coordination': 999,
        'For Sale': 0,
        'Sales Review': 0,
        'Sales Pipeline': 0,
        'Vacant Land': 999,
        'Sales Hold': 0,
        'Under Construction': 999,
        'Lease/ROE/TLA': 999,
        'Warehouse': 999,
        'NPOC Isolation - Do not move!': 999,
        'NSP Quarantine': 999,
        'Reconveyance Complete': 999,
        'Title Problem Identified': 999,
        'DJ Quarantine': 999,
        'PDD Quarantine': 999,
        'Intake Review': 999}

#if occupancy score > threshold, then no demo
dlba_school_df['Current_Status']\
[dlba_school_df['Occupancy_Model:_Probability_Pct'] > no_demo_occupancy_thresh] = 'Occupied'

#if MLB demo, then yes demo
dlba_school_df['Current_Status']\
[dlba_school_df['Non-City_Demo_Program'] =='MLB Demo'] = 'Demo Pipeline'

dlba_school_df['demo_decision'] = 0
dlba_school_df['demo_decision'] = [demo_decision_dict[i] for i in dlba_school_df["Current_Status"]]



dlba_school_df = dlba_school_df[dlba_school_df['demo_decision'] != 999]

dlba_school_df.head()

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school,demo_decision
0,001j000000KoyiZ,16032844.,16132 Cherrylawn,Formerly DLBA Owned,Residential-Structure,Auction,Occupied,NaN,9/13/2016,Fitzgerald/Marygrove,...,Fair,No,NaN,NaN,Secure,NaN,NaN,No,756.892564,0
1,001j000000Idx55,22079053.,17175 Faust,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Evergreen-Outer Drive,...,Good,No,NaN,No,NaN,NaN,NaN,No,1408.239825,0
2,001j000000Idvpv,22080009.,17620 Greenview,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Evergreen-Outer Drive,...,Good,No,NaN,No,NaN,NaN,NaN,No,1877.803304,0
3,001j000000IdzVm,22064522.,15359 Biltmore,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Crary/St Marys,...,Fair,No,NaN,No,Secure,NaN,NaN,No,1607.625307,0
4,001j000000Iem9O,21064817.,11080 Lakepointe,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Yorkshire Woods,...,Good,No,NaN,No,NaN,NaN,NaN,No,1859.498209,0


In [13]:
#code dummy variables

#mcm condition dummies
dlba_school_df['mcm_good_dummy'] = 0
dlba_school_df['mcm_good_dummy'] = [1 if i=='Good' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_fair_dummy'] = 0
dlba_school_df['mcm_fair_dummy'] = [1 if i=='Fair' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_poor_dummy'] = 0
dlba_school_df['mcm_poor_dummy'] = [1 if (i=='Poor' or i=='poor, fair') else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_demo_dummy'] = 0
dlba_school_df['mcm_demo_dummy'] = [1 if (i=='Suggest Demolition' or i=='suggest demolition, poor') else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_good_dummy'] = 0
dlba_school_df['mcm_good_dummy'] = [1 if i=='Good' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['dng_dummy'] = 0
dlba_school_df['dng_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DNG_Number']]

dlba_school_df['fire_dummy'] = 0
dlba_school_df['fire_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DFD_Fire:_Occurrence_Date']]

dlba_school_df['usps_vacant_dummy'] = 0
dlba_school_df['usps_vacant_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['USPS:_Vacancy_Indicated']]

dlba_school_df['foreclosed_dummy'] = 0
dlba_school_df['foreclosed_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['Foreclosure_Year']]

dlba_school_df['foreclosure_lag'] = 0
dlba_school_df['foreclosure_lag'] = [0 if pd.isnull(i) else (2017 - i) for i in dlba_school_df['Foreclosure_Year']]

dlba_school_df['boarding_needed_dummy'] = 0
dlba_school_df['boarding_needed_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Boarding_Needed']]

dlba_school_df['dumping_dummy'] = 0
dlba_school_df['dumping_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Dumping']]

#replace NaNs with zeros
dlba_school_df['Occupancy_Model:_Probability_Pct'] = [0 if pd.isnull(i) else i for i in dlba_school_df['Occupancy_Model:_Probability_Pct']]

#remove NaNs for Neighborhood_Sales_Median
#in final roll out I need to improve this!!!
dlba_school_df = dlba_school_df[~pd.isnull(dlba_school_df['Neighborhood_Sales_Median'])]

dlba_school_df_small = dlba_school_df.drop(['Account_ID', 'Property_Ownership', 'Property_Class', 'Program', \
                                            'Current_Status', 'Current_Status_Date', 'Neighborhood', 'DNG_Number', \
                                            'WCT_Foreclosure_Status', \
                                            'MCM_Blext_Created_At', 'MCM_Structure', 'MCM_Occupancy',  'MCM_Use',\
                                           'MCM_Units', 'MCM_Residence_Type', 'MCM_Commercial_Use', 'MCM_Industrial_Use',\
                                           'MCM_Institutional_Use', 'MCM_Public_Use', 'MCM_Level_of_Fire_Damage', \
                                            'MCM_Trespass', 'MCM_Vacant_Lot', 'MCM_Maintained', 'DFD_Fire:_Occurrence_Date',\
                                           'USPS:_Vacancy_Indicated', 'Foreclosure_Year', 'MCM_Fire_Damage',\
                                           'USPS:_Last_Update', 'Occupancy_Model:_Last_Update', 'MCM_Boarding_Needed',\
                                           'MCM_Dumping', 'MCM_Condition'], 1)
dlba_school_df_small.head(5)

,Parcel_ID,Account_Name,Non-City_Demo_Program,Neighborhood_Sales_Median,Occupancy_Model:_Probability_Pct,dist_to_school,demo_decision,mcm_good_dummy,mcm_fair_dummy,mcm_poor_dummy,mcm_demo_dummy,dng_dummy,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy
0,16032844.,16132 Cherrylawn,NaN,3.00,96.25,756.892564,0,0,1,0,0,0,0,1,0,0.0,0,0
1,22079053.,17175 Faust,NaN,9.52,96.60,1408.239825,0,1,0,0,0,0,0,1,0,0.0,0,0
2,22080009.,17620 Greenview,NaN,9.52,98.23,1877.803304,0,1,0,0,0,0,0,1,1,6.0,0,0
3,22064522.,15359 Biltmore,NaN,8.49,96.82,1607.625307,0,0,1,0,0,0,0,1,0,0.0,0,0
4,21064817.,11080 Lakepointe,NaN,2.21,99.55,1859.498209,0,1,0,0,0,0,0,1,0,0.0,0,0


In [14]:
dlba_school_df_small = dlba_school_df_small.rename(columns={'Occupancy_Model:_Probability_Pct': 'occupancy_model_pct'})
dlba_school_df_small = dlba_school_df_small[['Parcel_ID', 'Account_Name', 'demo_decision', 'Neighborhood_Sales_Median',\
                                            'occupancy_model_pct', 'dist_to_school',\
                                            'fire_dummy', 'usps_vacant_dummy', 'foreclosed_dummy',\
                                            'foreclosure_lag', 'boarding_needed_dummy', 'dumping_dummy', 'dng_dummy',\
                                            'mcm_good_dummy', 'mcm_fair_dummy', 'mcm_demo_dummy']]

#fit model 
y = dlba_school_df_small.demo_decision
X = dlba_school_df_small.iloc[:,3:]
'''
let's try this without the crossvalidation loop first


for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    #learn logistic regression over the training sample
    logit = LogisticRegression(C=1)
    logit.fit(X_train, y_train)
    # C here controls the regularization strength--which will be discussed in Bayesian session
    # For now just keep in mind that setting a high value will make the regularization ineffective

    #apply to the validation sample
    labelPred = logit.predict(X_test)
    #labelPred = logPred > 0.5
    labelActual = y_test
    
'''    


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)
    
#learn logistic regression over the training sample
logit = LogisticRegression(C=1)
logit.fit(X_train, y_train)
# C here controls the regularization strength--which will be discussed in Bayesian session
# For now just keep in mind that setting a high value will make the regularization ineffective

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
r_tree = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
r_tree.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [16]:
r_tree.feature_importances_

array([ 0.11185337,  0.2553302 ,  0.14947449,  0.00469798,  0.003891  ,
        0.02147761,  0.04425323,  0.06157179,  0.03133395,  0.12973251,
        0.04663578,  0.04021758,  0.0995305 ])

In [17]:
Feature_importance=pd.DataFrame([list(X_train.columns),list(r_tree.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]
Feature_importance.sort_values(by="importance",ascending=False)

,variables,importance
1,occupancy_model_pct,0.25533
2,dist_to_school,0.149474
9,dng_dummy,0.129733
0,Neighborhood_Sales_Median,0.111853
12,mcm_demo_dummy,0.0995305
7,boarding_needed_dummy,0.0615718
10,mcm_good_dummy,0.0466358
6,foreclosure_lag,0.0442532
11,mcm_fair_dummy,0.0402176
8,dumping_dummy,0.031334


# Accuracy Models

### logit model

In [18]:
labelActual = y_test

In [19]:
#apply to the validation sample
labelPred = logit.predict(X_test)
X_test_copy = X_test.copy()
X_test_copy['logit_predict'] = logit.predict(X_test)
sum(labelPred)

3149

In [20]:
len(y_test)

43237

In [21]:
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))

#compute precision, accuracy and recall
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
Prec= 100.0 * TP/(TP+FP)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))

Precision = 79.68%
Accuracy = 96.59%
Recall = 75.05%


### r_tree model

In [22]:
len(X_test)

43237

In [23]:
sum(labelActual)

3343

In [24]:
#apply to the validation sample
labelPred = r_tree.predict(X_test)
X_test_copy['r_tree_predict'] = r_tree.predict(X_test)
X_test_copy['labelActual'] = y_test
sum(labelPred)

3333

In [25]:
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))

#compute precision, accuracy and recall
Prec= 100.0 * TP/(TP+FP)
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))

Precision = 78.13%
Accuracy = 96.60%
Recall = 77.89%


In [26]:
### merge on address
dlba_school_df_address = dlba_school_df_small[['Account_Name']]
test_data_full = X_test_copy.merge(dlba_school_df_address, how='left', left_index=True, right_index=True)

In [27]:
test_data_full[((test_data_full.r_tree_predict==1) & (test_data_full.labelActual==0)) | ((test_data_full.r_tree_predict==0) & (test_data_full.labelActual==1))].head()

,Neighborhood_Sales_Median,occupancy_model_pct,dist_to_school,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy,dng_dummy,mcm_good_dummy,mcm_fair_dummy,mcm_demo_dummy,logit_predict,r_tree_predict,labelActual,Account_Name
132593,4.78,14.54,1516.870164,0,1,1,4.0,1,1,0,0,1,0,0,1,0,12630 Gunston
5418,0.84,14.54,5889.461347,0,1,1,7.0,0,1,0,0,0,0,1,0,1,8111 Bryden
2521,6.89,58.72,5977.433053,0,1,1,4.0,1,0,0,0,1,0,0,0,1,8617 Mark Twain
44087,1.70,14.54,4233.071723,0,1,1,4.0,0,0,0,0,0,0,1,0,1,11152 Corbett
130843,1.24,14.54,3947.995554,0,1,1,4.0,0,0,0,0,1,0,0,1,0,9375 Cheyenne
